In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import optuna


In [7]:

data = pd.read_csv('data/cleaned_dataset4.csv')

Mounted at /content/drive


In [8]:
# Convertir columnas booleanas a enteros (0 y 1)
bool_columns = data.select_dtypes(include=['bool']).columns
data[bool_columns] = data[bool_columns].astype(int)

# Comprobar y convertir a numérico
data = data.apply(pd.to_numeric, errors='coerce')
data = data.dropna()  # Eliminar filas con NaN

In [9]:
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values
y = y.flatten()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
def objective(trial):
    n_layers = trial.suggest_int('n_layers', 1, 3)  # Número de capas ocultas
    n_units = trial.suggest_int('n_units', 4, 128)   # Número de neuronas por capa
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'sigmoid'])  # Funciones de activación
    dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)  # Tasa de dropout
    batch_size = trial.suggest_int('batch_size', 16, 128)  # Tamaño del lote
    epochs = trial.suggest_int('epochs', 5, 50)  # Número de épocas

    # Crear el modelo
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))

    for _ in range(n_layers):
        model.add(keras.layers.Dense(n_units, activation=activation))
        model.add(keras.layers.Dropout(dropout_rate))

    model.add(keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=0)

    accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
    return accuracy


study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

print("Mejores hiperparámetros: ", study.best_params)
print("Mejor precisión: ", study.best_value)

[I 2024-10-09 10:53:30,283] A new study created in memory with name: no-name-d572d8b4-347d-474c-83b6-41dee2b54d0b
[I 2024-10-09 10:53:40,379] Trial 0 finished with value: 0.8173178434371948 and parameters: {'n_layers': 2, 'n_units': 7, 'activation': 'relu', 'dropout_rate': 0.1466502833520446, 'batch_size': 78, 'epochs': 26}. Best is trial 0 with value: 0.8173178434371948.
[I 2024-10-09 10:53:54,380] Trial 1 finished with value: 0.8738120198249817 and parameters: {'n_layers': 2, 'n_units': 83, 'activation': 'tanh', 'dropout_rate': 0.04321445191845408, 'batch_size': 101, 'epochs': 46}. Best is trial 1 with value: 0.8738120198249817.
[I 2024-10-09 10:54:06,642] Trial 2 finished with value: 0.8062301874160767 and parameters: {'n_layers': 1, 'n_units': 29, 'activation': 'tanh', 'dropout_rate': 0.33831427894435867, 'batch_size': 74, 'epochs': 38}. Best is trial 1 with value: 0.8738120198249817.
[I 2024-10-09 10:54:12,264] Trial 3 finished with value: 0.8321013450622559 and parameters: {'n_la